In [1]:
import os 
import pandas as pd 
import numpy as np

In [2]:
import librosa

In [3]:
def list_sample(dir):
    samples_dir, label= [], []
    for i in os.listdir(dir): 
        for j in os.listdir(os.path.join(dir, i)): 
            samples_dir.append(os.path.join(dir, i + f'/{j}'))
            label.append(i)
    return samples_dir, label

In [14]:
samples_dir, label= list_sample('genres_original/')

In [15]:
df = pd.DataFrame()
df['dir'] = samples_dir
df['label']= label

In [3]:
genres = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4, 
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [44]:
signal, sr= librosa.load('Shut Down.mp3')

In [ ]:
import numpy as np

def normalize(x):
    """Normalize a matrix x to have values between 0 and 1"""
    x_min = np.min(x)
    x_max = np.max(x)
    x_norm = (x - x_min) / (x_max - x_min)
    return x_norm

In [4]:
def preprocessing(dir, sr= 44100):
    df= pd.read_csv(dir)
    
    frame_length= int(sr * 0.3)
    hop_length= int(frame_length * 0.5) 
    
    arr_specs = []
    arr_genres = []
    
    for i in range(df.shape[0]): 
        try:
            samples, sr= librosa.load(df.loc[i, 'dir'], sr= sr)
            mel= librosa.feature.melspectrogram(y= samples, sr= sr, n_mels= 128)
            mel= librosa.amplitude_to_db(mel)
            mel= normalize(mel)

            feature= librosa.util.frame(mel, frame_length= 128, hop_length= 64).T
            
            arr_specs.append(feature)
            arr_genres.append([genres[df.loc[i, 'label']]] * feature.shape[0])
        except:
            print(f'File {df.loc[i, "dir"]} error!!')
    return arr_specs, arr_genres

In [5]:
X, Y= preprocessing('dir_samples.csv', sr= 44100)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11996\60078141.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr= librosa.load(df.loc[i, 'dir'], sr= sr)
C:\Users\Admin\anaconda3\envs\torch\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


File genres_original/jazz/jazz.00054.wav error!!


In [32]:
np.concatenate(X).shape , np.concatenate(Y).shape

In [37]:
np.save('x_gtzan.npy', np.concatenate(X))
np.save('y_gtzan.npy', np.concatenate(Y))